In [279]:
import numpy as np
import torch
import math
import importlib
# import socialSigLayers
# importlib.reload(socialSigLayers)
import pandas as pd
import random

In [280]:
class bilinearImputation(torch.nn.Module):
    '''
    Class to create the social signature image
    '''
    def __init__(self, X):
        super(bilinearImputation, self).__init__()
        self.W = torch.nn.Parameter(torch.tensor(np.arange(0,X.shape[1]), dtype = torch.float32, requires_grad=True))
        self.outDim = [10,10]
        self.inDim = math.ceil(math.sqrt(X.shape[1]))

    def forward(self, batchX):
        
        #print("    W at beginning: ", self.W) 
        self.X = batchX
        xTemp = torch.stack([self.X, self.W.repeat(self.X.shape[0],1).data])
        XSort = torch.sort(xTemp, dim=2, descending=False)
        inDataSize = XSort[0][0].shape[1] #Data we have per dimension
        targetSize = self.inDim ** 2
        paddingOffset = targetSize - inDataSize
        paddedInX = torch.nn.functional.pad(input=XSort[0][0], pad=(0,paddingOffset), mode="constant", value=0)
        buildImage = torch.reshape(paddedInX,(self.X.shape[0], 1, self.inDim, self.inDim))   
        return torch.nn.functional.interpolate(buildImage, size=([self.outDim[0], self.outDim[1]]), mode='bilinear')

In [281]:
###### Define our model
class SocialSigNet(torch.nn.Module):
    def __init__(self, X):
        super().__init__()
        self.SocialSig = bilinearImputation(X=X)                
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(10,10))  #10,10 is static here.  Will need to be dynamic based on user dim settings.
        self.norm = torchvision.transforms.Normalize(0.5, 0.5)
        
        
    def forward(self, X):
        out = self.SocialSig(X)
        out = self.maxPool(out)       
        return out

In [282]:
####### Load our Data
devSet = pd.read_csv("us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)

In [283]:
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values
####### Build and fit the Model
model = SocialSigNet(X=X)
lr = 1
batchSize = 8

In [284]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [285]:
def update_function(param, grad, loss, learning_rate):
    return param - learning_rate * grad.mean(axis = 0)

In [286]:
for t in range(5):
    #Batches
    batchObs = random.sample(range(0, len(y)), batchSize)
    modelX = X[batchObs]
    modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
    modely = torch.tensor(y[batchObs], dtype = torch.float32)  # MADE A CHANGE HERE 
    
    
    print("EPOCH: ", t)
    y_pred = model(modelX)
    loss = criterion(y_pred, modely)    
    print("    Loss:     ", loss)
    print(loss)
    
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    grad = torch.autograd.grad(outputs=loss, inputs=modelX, retain_graph = True)
    loss.backward()
    optimizer.step()
    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
    with torch.no_grad():
        for p in model.parameters():
            print("    In with:        ", p.data)
            new_val = update_function(p, grad[0], loss, lr)
            p.copy_(new_val)
    
    print("\n")

EPOCH:  0
    Loss:      tensor(1.2220e+16, grad_fn=<MseLossBackward>)
tensor(1.2220e+16, grad_fn=<MseLossBackward>)
    In with:         tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29.])


EPOCH:  1
    Loss:      tensor(2.1180e+17, grad_fn=<MseLossBackward>)
tensor(2.1180e+17, grad_fn=<MseLossBackward>)
    In with:         tensor([ 0.0000e+00,  1.0000e+00,  2.0000e+00,  3.0000e+00,  4.0000e+00,
         5.0000e+00,  6.0000e+00,  7.0000e+00,  8.0000e+00,  9.0000e+00,
         1.0000e+01,  1.1000e+01,  1.2000e+01,  1.3000e+01,  1.4000e+01,
         1.5000e+01,  1.6000e+01,  1.7000e+01,  1.8000e+01, -1.8088e+08,
         2.0000e+01,  2.1000e+01,  2.2000e+01,  2.3000e+01,  2.4000e+01,
         2.5000e+01,  2.6000e+01,  2.7000e+01,  2.8000e+01,  2.9000e+01])


EPOCH:  2
    Loss:      tensor(2.5730e+15, grad_fn=<MseLossBackward>)
tensor(2.5730e+15, grad_fn=<MseLoss

/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return

In [287]:
print("Model.parameters")
print(model.parameters)
print("SSParam")
print(list(model.SocialSig.parameters()))
print("is_leaf")
print(list(model.SocialSig.parameters())[0].is_leaf)
print("gradfn")
print(list(model.SocialSig.parameters())[0].grad_fn)
print("Grad")
print(list(model.SocialSig.parameters())[0].grad)

Model.parameters
<bound method Module.parameters of SocialSigNet(
  (SocialSig): bilinearImputation()
  (maxPool): MaxPool2d(kernel_size=(10, 10), stride=(10, 10), padding=0, dilation=1, ceil_mode=False)
)>
SSParam
[Parameter containing:
tensor([ 0.0000e+00,  1.0000e+00,  2.0000e+00,  3.0000e+00,  4.0000e+00,
         5.0000e+00,  6.0000e+00,  7.0000e+00,  8.0000e+00,  9.0000e+00,
         1.0000e+01,  1.1000e+01,  1.2000e+01,  1.3000e+01,  1.4000e+01,
         1.5000e+01,  1.6000e+01,  1.7000e+01,  1.8000e+01, -8.0778e+08,
         2.0000e+01,  2.1000e+01,  2.2000e+01,  2.3000e+01,  2.4000e+01,
         2.5000e+01,  2.6000e+01,  2.7000e+01,  2.8000e+01,  2.9000e+01],
       requires_grad=True)]
is_leaf
True
gradfn
None
Grad
None
